# folium - 지도 시각화 도구

In [14]:
!pip install folium

In [1]:
import folium
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### - 서울 지역의 구별 boundary 시각화

In [2]:
# 서울 행정구역 json raw파일
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [3]:
m = folium.Map(
    location=[37.559819, 127.063895],
    zoom_start=11, 
    tiles='cartodbpositron' # tiles 의 default는 OpenStreetMap이며, Stamen Toner, cartodbpositron 등으로 설정 가능
)

folium.GeoJson(
    seoul_geo,
#     name='지역구'  ??
).add_to(m)

m

In [4]:
m = folium.Map(
    location=[37.559819, 127.063895],
    zoom_start=11, 
    tiles='OpenStreetMap'
)

folium.GeoJson(
    seoul_geo,
#     name='지역구'  ??
).add_to(m)

m

In [5]:
m = folium.Map(
    location=[37.559819, 127.063895],
    zoom_start=11, 
    tiles='Stamen Toner' 
)

folium.GeoJson(
    seoul_geo,
#     name='지역구'  ??
).add_to(m)

m

### - 데이터프레임을 활용하여 서울 자치구별 데이터 시각화

#### 1. 서울 자치구별 버스 정류장 수

In [6]:
seoul_bus=pd.read_csv('bus_station_boarding_month_202204.csv', sep=',', encoding = 'cp949')
seoul_bus

,노선번호,버스정류장ARS번호,역명
0,2112,06243,장안동삼성쉐르빌아파트
1,2112,06242,새서울병원
2,2112,06238,장안동근린공원
3,2112,06309,휘경공고휘경주공아파트
4,100,11428,한성여객종점
...,...,...,...
1169873,7738,12004,수색교
1169874,201,06278,세종대왕기념관
1169875,강서01,16555,화곡본동시장
1169876,3321,25236,굽은다리사거리


In [7]:
#버스정류장ARS번호 앞 두자리는 정류장이 위치한 구를 의미함 (01~25)
seoul_bus['자치구'] = seoul_bus['버스정류장ARS번호'].str.slice(start=0, stop=2)
seoul_bus['자치구'] = seoul_bus['자치구'].map({
    '01': '종로구',
    '02': '중구',
    '03': '용산구',
    '04': '성동구',
    '05': '광진구',
    '06': '동대문구',
    '07': '중랑구',
    '08': '성북구',
    '09': '강북구',
    '10': '도봉구',
    '11': '노원구',
    '12': '은평구',
    '13': '서대문구',
    '14': '마포구',
    '15': '양천구',
    '16': '강서구',
    '17': '구로구',
    '18': '금천구',
    '19': '영등포구',
    '20': '동작구',
    '21': '관악구',
    '22': '서초구',
    '23': '강남구',
    '24': '송파구',
    '25': '강동구'})

# (가상기점),(가상종점)때문에 ARS번호가 '~'로 나오는 곳이 있는데, 이를 제거함
seoul_bus.loc[seoul_bus['버스정류장ARS번호'] == '~']
seoul_bus = seoul_bus.dropna()

seoul_bus = seoul_bus.astype({'버스정류장ARS번호':'int'})

In [8]:
only_seoul = pd.read_csv('bus_station.csv', sep=',', encoding = 'UTF-8')
only_seoul.rename(columns={'ARS-ID':'버스정류장ARS번호'}, inplace=True)
df_seoul_bus_station = pd.merge(only_seoul,seoul_bus, how='inner', on='버스정류장ARS번호')
df_seoul_bus_station

,버스정류장ARS번호,정류소명,노선번호,역명,자치구
0,1001,종로2가사거리,N37,종로2가사거리,종로구
1,1001,종로2가사거리,N37,종로2가사거리,종로구
2,1001,종로2가사거리,741,종로2가사거리,종로구
3,1001,종로2가사거리,470,종로2가사거리,종로구
4,1001,종로2가사거리,741,종로2가사거리,종로구
...,...,...,...,...,...
1060881,25990,암사삼성.현대아파트,강동02,암사삼성.현대아파트,강동구
1060882,25990,암사삼성.현대아파트,강동02,암사삼성.현대아파트,강동구
1060883,25990,암사삼성.현대아파트,강동02,암사삼성.현대아파트,강동구
1060884,25990,암사삼성.현대아파트,강동02,암사삼성.현대아파트,강동구


In [9]:
df_seoul_bus_station= df_seoul_bus_station.groupby(by=['자치구'], as_index=True)['버스정류장ARS번호'].nunique()
df_seoul_bus_station

자치구
강남구     499
강동구     369
강북구     413
강서구     566
관악구     466
광진구     269
구로구     486
금천구     345
노원구     514
도봉구     359
동대문구    306
동작구     438
마포구     562
서대문구    456
서초구     600
성동구     435
성북구     602
송파구     415
양천구     319
영등포구    465
용산구     323
은평구     501
종로구     349
중구      178
중랑구     380
Name: 버스정류장ARS번호, dtype: int64

In [10]:
m = folium.Map(
    location=[37.559819, 127.063895],
    zoom_start=11, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
#     name='지역구'  ??
).add_to(m)

m.choropleth(geo_data=seoul_geo,
             data=df_seoul_bus_station,
             fill_color='YlOrRd',
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name='각 구별 정류장수'
            )
m.save('seoul_bus_station.html')
m

C:\Users\User\anaconda3\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


#### 2. 서울 자치구별 택시운송업 종사자수

In [11]:
seoul_business=pd.read_csv('seoul_business_type.csv',sep=',',encoding='UTF-8', header=1)
seoul_business.drop(index=[0,1],axis=0,inplace=True)
seoul_business.reset_index(drop=True, inplace=True)
seoul_business=seoul_business.loc[seoul_business['동']=='소계',:]
seoul_business.drop(columns=['동'],axis=1,inplace=True)
seoul_business.reset_index(drop=True, inplace=True)

tmp1=seoul_business[['자치구']]
tmp1.reset_index(drop=False,inplace=True)
tmp2=seoul_business.drop(columns='자치구', axis=1)
# 정규식을 통한 object->int 형변환
tmp2=tmp2.replace('(.*),(.*)', r'\1\2', regex=True)
tmp2=tmp2.astype(int)
tmp2.reset_index(drop=False,inplace=True)
tmp2['합계']=tmp2.sum(axis=1)
seoul_business=pd.merge(tmp1,tmp2,how='inner',on='index')
seoul_business.drop(columns='index',axis=1,inplace=True)

seoul_business.rename(columns={ '택시운송업' : '택시운송업 사업체수', '택시운송업.1' : '택시운송업 종사자수',
                                '한식 일반 음식점업' : '한식 일반 음식점업 사업체수', '한식 일반 음식점업.1' : '한식 일반 음식점업 종사자수',
                                '용달 화물자동차 운송업' : '용달 화물자동차 운송업 사업체수', '용달 화물자동차 운송업.1' : '용달 화물자동차 운송업 종사자수',
                                '부동산 중개 및 대리업' : '부동산 중개 및 대리업 사업체수', '부동산 중개 및 대리업.1' : '부동산 중개 및 대리업 종사자수',
                                '두발미용업' : '두발미용업 사업체수', '두발미용업.1' : '두발미용업 종사자수',
                                '커피전문점' : '커피전문점 사업체수', '커피전문점.1' : '커피전문점 종사자수',     
                                '남녀용 겉옷 및 셔츠 도매업' : '남녀용 겉옷 및 셔츠 도매업 사업체수', '남녀용 겉옷 및 셔츠 도매업.1': '남녀용 겉옷 및 셔츠 도매업 종사자수',
                                '기타주점업' : '기타주점업 사업체수', '기타주점업.1' : '기타주점업 종사자수',          
                                '일반 교과 학원' : '일반 교과 학원 사업체수', '일반 교과 학원.1' : '일반 교과 학원 종사자수',
                                '한식 육류요리 전문점' : '한식 육류요리 전문점 사업체수', '한식 육류요리 전문점.1' : '한식 육류요리 전문점 종사자수'}, inplace=True)

seoul_business

,자치구,택시운송업 사업체수,택시운송업 종사자수,한식 일반 음식점업 사업체수,한식 일반 음식점업 종사자수,용달 화물자동차 운송업 사업체수,용달 화물자동차 운송업 종사자수,부동산 중개 및 대리업 사업체수,부동산 중개 및 대리업 종사자수,두발미용업 사업체수,...,커피전문점 종사자수,남녀용 겉옷 및 셔츠 도매업 사업체수,남녀용 겉옷 및 셔츠 도매업 종사자수,기타주점업 사업체수,기타주점업 종사자수,일반 교과 학원 사업체수,일반 교과 학원 종사자수,한식 육류요리 전문점 사업체수,한식 육류요리 전문점 종사자수,합계
0,종로구,215,215,1548,5912,618,646,549,889,261,...,3613,400,962,537,1413,110,382,437,1982,22117
1,중구,230,235,1588,6414,300,333,576,1152,339,...,3542,10810,24248,381,982,57,175,443,2121,55612
2,용산구,416,523,857,2987,455,466,791,1214,366,...,2076,54,317,487,1332,104,573,300,1184,15915
3,성동구,1200,2527,956,2785,1072,1161,801,1453,489,...,1859,149,1572,285,621,258,1494,400,1545,22028
4,광진구,1119,1744,963,2861,968,985,871,1313,746,...,1887,89,687,633,1577,413,1646,467,1678,22578
5,동대문구,1546,2093,1236,3566,1431,1456,842,1208,611,...,1472,211,976,499,1082,302,984,442,1509,22856
6,중랑구,3634,6262,991,2479,1804,1877,738,1104,790,...,1037,63,284,535,1000,288,1045,499,1504,27429
7,성북구,1835,2643,973,2822,1312,1335,846,1175,739,...,1384,85,454,512,1030,464,1755,365,1334,22772
8,강북구,1713,2589,965,2663,1024,1030,589,867,673,...,1095,40,94,580,1275,220,740,362,1176,19058
9,도봉구,2993,5783,672,2009,1571,1574,515,794,600,...,837,22,43,258,477,351,1162,302,943,22050


In [12]:
df_seoul_business=seoul_business[['자치구','택시운송업 종사자수']]
df_seoul_business=df_seoul_business.groupby(by='자치구')['택시운송업 종사자수'].sum()
df_seoul_business

자치구
강남구     2517
강동구     4980
강북구     2589
강서구     6922
관악구     2027
광진구     1744
구로구     3817
금천구     2598
노원구     6074
도봉구     5783
동대문구    2093
동작구     1115
마포구     1838
서대문구    2677
서초구      903
성동구     2527
성북구     2643
송파구     4850
양천구     4942
영등포구    2397
용산구      523
은평구     4430
종로구      215
중구       235
중랑구     6262
Name: 택시운송업 종사자수, dtype: int32

In [13]:
m = folium.Map(
    location=[37.559819, 127.063895],
    zoom_start=11, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
#     name='지역구'  ??
).add_to(m)

m.choropleth(geo_data=seoul_geo,
             data=df_seoul_business,
             fill_color='YlOrRd',
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name='각 구별 택시운송업 종사자수'
            )

m.save('seoul_taxi_driver.html')
m

C:\Users\User\anaconda3\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


### 참고 및 출처

##### -  서울 행정구역 json raw파일

https://github.com/southkorea/southkorea-maps

위 github에서 대한민국 시도, 읍면동 기준으로 boundary 된 json 파일을 얻을 수 있다.

추가적으로 대한민국 행정구역 경계 및 json 파일 변환 및 다운받는 방법은 여기(https://yeomss.tistory.com/267) 참고해서 해보길...

##### - folium 사용법

https://teddylee777.github.io/visualization/folium

##### - 서울 열린데이터 광장 
